In [3]:
!nvidia-smi

Wed Apr  2 01:59:29 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     On   | 00000000:12:00.0 Off |                  N/A |
| 52%   65C    P8    11W / 125W |      1MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 4000     On   | 00000000:86:00.0 Off |                  N/A |
| 55%   

In [2]:
!kill 1763117      

/bin/bash: line 1: kill: (1763117) - No such process


In [3]:
import SimpleITK as sitk
import nibabel as nib
import numpy as np
import os
import matplotlib.pyplot as plt
# import torch
# from torch.utils.data import Dataset
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Name:", tf.config.list_physical_devices('GPU'))
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import plot_model, to_categorical
tf.debugging.set_log_device_placement(False)
tf.config.experimental_run_functions_eagerly(False)
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
tf.debugging.set_log_device_placement(False)
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold
# from torch.utils.data import DataLoader
from scipy.ndimage import zoom
from glob import glob
import re
from skimage import measure
import torchio as tio
import shutil
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

2025-04-01 20:26:46.015397: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 20:26:46.052431: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 20:26:46.052456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 20:26:46.052473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 20:26:46.065306: I tensorflow/core/platform/cpu_feature_g

TensorFlow Version: 2.14.0
Num GPUs Available: 2
GPU Name: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


/home/student11/anaconda3/envs/mlmed-huyproman/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def the_great_march_of_return(source_dir, target_dir): ## DO NOT USE THIS AGAIN
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for subset in ['training', 'validation', 'testing']:
        subset_src = os.path.join(source_dir, subset)
        subset_dst = os.path.join(target_dir, subset)

        if not os.path.exists(subset_dst):
            os.makedirs(subset_dst)

        for patient_folder in os.listdir(subset_src):
            patient_src = os.path.join(subset_src, patient_folder)
            patient_dst = os.path.join(subset_dst, patient_folder)

            if not os.path.isdir(patient_src):
                continue

            if not os.path.exists(patient_dst):
                os.makedirs(patient_dst)

            for file_name in os.listdir(patient_src):
                if file_name == 'MANDATORY_CITATION.md' or file_name.endswith('_4d.nii.gz'):
                    continue 

                src_file = os.path.join(patient_src, file_name)
                dst_file = os.path.join(patient_dst, file_name)

                if os.path.isfile(src_file):
                    shutil.copy2(src_file, dst_file)

            print(f'Copied {patient_folder} from {subset} successfully.')


source_dir = '/home/student11/ds/acdc/database'
target_dir = '/home/student11/ds/mlmed-huyproman/database'

the_great_march_of_return(source_dir, target_dir)

print('Dataset copying completed successfully!')

In [17]:
def get_folder_size(path='.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

size_bytes = get_folder_size('/home/student11/ds/mlmed-huyproman/cropped_slices_processed_352')
print(f"Folder size: {size_bytes / (1024**2):.2f} MB")

Folder size: 753.25 MB


In [2]:
import os
file_path = '/home/student11/ds/mlmed-huyproman/cropped_slices_processed.zip'
file_size = os.path.getsize(file_path)
print(f"File đang là {file_size/1024**2:.4f}MB")

File đang là 901.2329MB


In [ ]:
# import zipfile
# zip_path = '/home/student11/ds/mlmed-huyproman/cropped_slices_processed.zip'
# extract_to = os.path.dirname(zip_path)  

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to)

# print(f"Extracted to: {extract_to}")

Extracted to: /home/student11/ds/mlmed-huyproman


---

In [4]:
PROCESSED_DIR = "/home/student11/ds/mlmed-huyproman/cropped_slices_processed_352"

for dataset in ["training", 'validation', 'testing']:
    dataset_path = os.path.join(PROCESSED_DIR, dataset)
    print(f"\n📁 Dataset: {dataset}\n{'-' * 40}")
    
    for patient_folder in os.listdir(dataset_path):
        patient_path = os.path.join(dataset_path, patient_folder)
        
        if not os.path.isdir(patient_path):
            continue

        for file in os.listdir(patient_path):
            if file.endswith(".nii.gz"):
                file_path = os.path.join(patient_path, file)
                
                try:
                    img = nib.load(file_path)
                    voxel_spacing = img.header.get_zooms()
                    shape = img.shape
                    print(f"- {file_path}")
                    print(f"  🧊 Spacing: {voxel_spacing}")
                    print(f"  📐 Shape: {shape}\n")
                except Exception as e:
                    print(f"❌ Error loading {file_path}: {e}")


📁 Dataset: training
----------------------------------------
- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/training/patient025/patient025_ES_gt_processed.nii.gz
  🧊 Spacing: (1.0, 1.0, 1.0)
  📐 Shape: (224, 224, 5)

- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/training/patient025/patient025_ED_gt_processed.nii.gz
  🧊 Spacing: (1.0, 1.0, 1.0)
  📐 Shape: (224, 224, 7)

- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/training/patient025/patient025_ED_processed.nii.gz
  🧊 Spacing: (1.0, 1.0, 1.0)
  📐 Shape: (224, 224, 7)

- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/training/patient025/patient025_ES_processed.nii.gz
  🧊 Spacing: (1.0, 1.0, 1.0)
  📐 Shape: (224, 224, 5)

- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/training/patient029/patient029_ES_gt_processed.nii.gz
  🧊 Spacing: (1.0, 1.0, 1.0)
  📐 Shape: (224, 224, 7)

- /home/student11/ds/mlmed-huyproman/cropped_slices_processed_352/tra

In [ ]:
## OLD UNET DO NOT RUN

def conv_block(x, filters):
        x = Conv2D(filters, (3, 3), padding="same", kernel_initializer = 'he_normal')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.01)(x)
        x = Conv2D(filters, (3, 3), padding="same", kernel_initializer = 'he_normal')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.01)(x)
        return x
def upsample_block(x, conv_features, filters):
    x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same")(x)
    x = concatenate([x, conv_features])
    x = conv_block(x, filters)
    return x
def unet(input_shape=(224, 224, 1), num_classes=4):
    inputs = Input(input_shape)

    c1 = conv_block(inputs, 26) 
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 52)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 104)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 208)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv_block(p4, 416) 

    u6 = upsample_block(c5, c4, 208)
    u7 = upsample_block(u6, c3, 104)
    u8 = upsample_block(u7, c2, 52)
    u9 = upsample_block(u8, c1, 26)

    outputs = Conv2D(num_classes, (1, 1), activation="softmax")(u9)

    model = Model(inputs, outputs, name="UNet_type_shit")
    return model

model = unet()
model.summary()

2025-04-01 20:26:51.222930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6653 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:12:00.0, compute capability: 7.5
2025-04-01 20:26:51.223728: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6653 MB memory:  -> device: 1, name: Quadro RTX 4000, pci bus id: 0000:86:00.0, compute capability: 7.5


Model: "UNet_type_shit"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 32)         320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 32)         0         ['batch_normaliza

In [6]:
## ResAttASPP-UNet DO NOT RUN

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, LeakyReLU, Add, Multiply, Activation
from tensorflow.keras.models import Model

# Residual Convolution Block
def residual_conv_block(x, filters):
    shortcut = Conv2D(filters, (1, 1), padding="same", kernel_initializer='he_normal')(x)
    x = Conv2D(filters, (3, 3), padding="same", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Conv2D(filters, (3, 3), padding="same", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    x = LeakyReLU(alpha=0.01)(x)
    return x

# Attention Gate
def attention_gate(x, g, filters):
    theta_x = Conv2D(filters, (1, 1), kernel_initializer='he_normal')(x)
    phi_g = Conv2D(filters, (1, 1), kernel_initializer='he_normal')(g)
    add = Add()([theta_x, phi_g])
    act = Activation('relu')(add)
    psi = Conv2D(1, (1, 1), kernel_initializer='he_normal')(act)
    sigmoid = Activation('sigmoid')(psi)
    return Multiply()([x, sigmoid])

# ASPP Block
def aspp_block(x, filters):
    conv1 = Conv2D(filters, 1, padding='same', dilation_rate=1, kernel_initializer='he_normal')(x)
    conv6 = Conv2D(filters, 3, padding='same', dilation_rate=6, kernel_initializer='he_normal')(x)
    conv12 = Conv2D(filters, 3, padding='same', dilation_rate=12, kernel_initializer='he_normal')(x)
    conv18 = Conv2D(filters, 3, padding='same', dilation_rate=18, kernel_initializer='he_normal')(x)
    x = concatenate([conv1, conv6, conv12, conv18])
    x = Conv2D(filters, 1, padding='same', kernel_initializer='he_normal')(x)
    return x

# Upsample Block with Attention
def upsample_block(x, conv_features, filters):
    x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same", kernel_initializer='he_normal')(x)
    attended_features = attention_gate(conv_features, x, filters)
    x = concatenate([x, attended_features])
    x = residual_conv_block(x, filters)
    return x

# ResAttASPP-UNet
def ResAttASPP_unet(input_shape=(224, 224, 1), num_classes=4):
    inputs = Input(input_shape)

    # Encoder
    c1 = residual_conv_block(inputs, 32)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = residual_conv_block(p1, 64)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = residual_conv_block(p2, 128)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = residual_conv_block(p3, 256)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = residual_conv_block(p4, 416)

    # ASPP after bottleneck
    aspp = aspp_block(c5, 256)

    # Decoder with attention gates
    u6 = upsample_block(aspp, c4, 256)
    u7 = upsample_block(u6, c3, 128)
    u8 = upsample_block(u7, c2, 64)
    u9 = upsample_block(u8, c1, 32)

    outputs = Conv2D(num_classes, (1, 1), activation="softmax", kernel_initializer='he_normal')(u9)

    model = Model(inputs, outputs, name="ResAttASPP_UNet")
    return model

model = unet()
model.summary()

Model: "UNet_type_shit"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 224, 224, 32)         320       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 224, 224, 32)         128       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 leaky_re_lu_18 (LeakyReLU)  (None, 224, 224, 32)         0         ['batch_normaliza

In [7]:
## NO AUGMENT LAST FRAME
# def load_patient_slices(patient_folder, patient_id, num_classes=4):
#     images, masks = [], []

#     last_frame_index = get_last_frame(patient_folder, patient_id)
#     if last_frame_index is None:
#         return [], []

#     for frame_idx in [1, last_frame_index]:  
#         frame_str = f'frame{str(frame_idx).zfill(2)}'

#         image_path = os.path.join(patient_folder, f'patient{patient_id}_{frame_str}.nii.gz')
#         mask_path = os.path.join(patient_folder, f'patient{patient_id}_{frame_str}_gt.nii.gz')

#         if not os.path.exists(image_path) or not os.path.exists(mask_path):
#             print(f"⚠️ Missing file: {image_path} or {mask_path}")
#             continue  

#         image_nii = nib.load(image_path)
#         mask_nii = nib.load(mask_path)

#         image_array = image_nii.get_fdata()
#         mask_array = mask_nii.get_fdata()

#         image_array = (image_array - np.mean(image_array)) / (np.std(image_array) + 1e-10)

#         for i in range(image_array.shape[2]): 
#             img_slice = image_array[:, :, i]
#             mask_slice = mask_array[:, :, i]

#             img_slice = np.expand_dims(img_slice, axis=-1)

#             mask_slice = to_categorical(mask_slice, num_classes=num_classes).astype("float32")

#             images.append(img_slice)
#             masks.append(mask_slice)

#     return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)
# def get_last_frame(patient_folder, patient_id):
# #     nii_files = os.listdir(patient_folder)
    
# #     frame_numbers = [
# #         int(re.search(r'frame(\d+)', f).group(1))
# #         for f in nii_files if re.search(r'frame(\d+)', f) and "frame01" not in f
# #     ]
    
# #     if frame_numbers:
# #         return max(frame_numbers)  
# #     else:
# #         print(f"⚠️ No ES frame found for patient {patient_id}. Skipping ES processing.")
# #         return None  

In [ ]:
## AUGMENT OLD
# def load_patient_slices(patient_folder, num_classes=4, use_augmentations=True):
#     images, masks = [], []

#     patient_id = patient_folder[-3:]  

#     def load_pair(image_path, mask_path):
#         image_nii = nib.load(image_path)
#         mask_nii = nib.load(mask_path)

#         image_array = image_nii.get_fdata()
#         mask_array = mask_nii.get_fdata()

#         image_array = (image_array - np.mean(image_array)) / (np.std(image_array) + 1e-10)

#         for i in range(image_array.shape[2]): 
#             img_slice = np.expand_dims(image_array[:, :, i], axis=-1)
#             mask_slice = to_categorical(mask_array[:, :, i], num_classes=num_classes).astype("float32")

#             images.append(img_slice)
#             masks.append(mask_slice)

#     for phase in ["ED", "ES"]:
#         base_image = f"patient{patient_id}_{phase}_processed.nii.gz"
#         base_mask = f"patient{patient_id}_{phase}_gt_processed.nii.gz"

#         image_path = os.path.join(patient_folder, base_image)
#         mask_path = os.path.join(patient_folder, base_mask)

#         if os.path.exists(image_path) and os.path.exists(mask_path):
#             load_pair(image_path, mask_path)
#         else:
#             print(f"⚠️ Missing original: {image_path} or {mask_path}")

#         if use_augmentations:
#             for aug_idx in [0]:
#                 aug_image = f"patient{patient_id}_{phase}_processed_augmented{aug_idx}.nii.gz"
#                 aug_mask = f"patient{patient_id}_{phase}_gt_processed_augmented{aug_idx}.nii.gz"

#                 aug_image_path = os.path.join(patient_folder, aug_image)
#                 aug_mask_path = os.path.join(patient_folder, aug_mask)

#                 if os.path.exists(aug_image_path) and os.path.exists(aug_mask_path):
#                     load_pair(aug_image_path, aug_mask_path)
#                 else:
#                     print(f"⚠️ Missing augmented{aug_idx}: {aug_image_path} or {aug_mask_path}")

#     return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)

In [8]:
def load_patient_slices(patient_folder, num_classes=4):
    images, masks = [], []

    for phase in ["ED", "ES"]:
        image_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_processed.nii.gz")
        mask_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_gt_processed.nii.gz")

        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"⚠️ Missing file: {image_path} or {mask_path}")
            continue  

        image_nii = nib.load(image_path)
        mask_nii = nib.load(mask_path)

        image_array = image_nii.get_fdata()
        mask_array = mask_nii.get_fdata()

        image_array = (image_array - np.mean(image_array)) / (np.std(image_array) + 1e-10)

        for i in range(image_array.shape[2]): 
            img_slice = image_array[:, :, i]
            mask_slice = mask_array[:, :, i]

            img_slice = np.expand_dims(img_slice, axis=-1) 
            
            mask_slice = to_categorical(mask_slice, num_classes=num_classes).astype("float32")

            images.append(img_slice)
            masks.append(mask_slice)

    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)  


def load_dataset(root_dir, dataset_type):
    images, masks = [], []
    dataset_path = os.path.join(root_dir, dataset_type)

    for patient_folder in sorted(os.listdir(dataset_path)):
        patient_path = os.path.join(dataset_path, patient_folder)

        if os.path.isdir(patient_path):
            patient_id = patient_folder[-3:]
            patient_images, patient_masks = load_patient_slices(patient_path, num_classes = 4)
            print(f"✅ Loaded {len(patient_images)} images and {len(patient_masks)} masks from {patient_folder}")
            images.extend(patient_images)
            masks.extend(patient_masks)

    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)

In [9]:
X_train, Y_train = load_dataset("/home/student11/ds/mlmed-huyproman/cropped_slices_processed_352", "training")
X_val, Y_val = load_dataset('/home/student11/ds/mlmed-huyproman/cropped_slices_processed_352', 'validation')
# X_test, Y_test = load_dataset("/home/student11/ds/mlmed-huyproman/cropped_slices_processed", "testing")

print(f"✅ Training Data: {X_train.shape}, {Y_train.shape}")
print(f"✅ Validation Data: {X_val.shape}, {Y_val.shape}")
# print(f"✅ Testing Data: {X_test.shape}, {Y_test.shape}")

✅ Loaded 15 images and 15 masks from patient001
✅ Loaded 16 images and 16 masks from patient002
✅ Loaded 16 images and 16 masks from patient003
✅ Loaded 16 images and 16 masks from patient004
✅ Loaded 16 images and 16 masks from patient005
✅ Loaded 16 images and 16 masks from patient006
✅ Loaded 17 images and 17 masks from patient007
✅ Loaded 14 images and 14 masks from patient008
✅ Loaded 16 images and 16 masks from patient009
✅ Loaded 18 images and 18 masks from patient010
✅ Loaded 16 images and 16 masks from patient011
✅ Loaded 17 images and 17 masks from patient012
✅ Loaded 16 images and 16 masks from patient013
✅ Loaded 17 images and 17 masks from patient014
✅ Loaded 16 images and 16 masks from patient015
✅ Loaded 18 images and 18 masks from patient016
✅ Loaded 13 images and 13 masks from patient021
✅ Loaded 7 images and 7 masks from patient022
✅ Loaded 10 images and 10 masks from patient023
✅ Loaded 13 images and 13 masks from patient024
✅ Loaded 12 images and 12 masks from patie

In [10]:
print("Any NaNs in X_train?", np.isnan(X_train).any())
print("Any NaNs in Y_train?", np.isnan(Y_train).any())

Any NaNs in X_train? False


Any NaNs in Y_train? False


In [11]:
def dice_loss_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    
    for i in range(1, num_classes):
        y_true_class = K.flatten(y_true[..., i])
        y_pred_class = K.flatten(y_pred[..., i])

        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)

    return 1 - K.mean(K.stack(dice_scores), axis=0)  

def dice_score_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    
    for i in range(1, num_classes):
        y_true_class = K.flatten(y_true[..., i])
        y_pred_class = K.flatten(y_pred[..., i])

        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)

    return K.mean(K.stack(dice_scores), axis=0)  

def dice_metric(y_true, y_pred):
    return dice_score_per_class(y_true, y_pred, num_classes=4)

def dice_score_each_individual_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []

    for i in range(num_classes):
        y_true_class = tf.keras.backend.flatten(y_true[..., i])
        y_pred_class = tf.keras.backend.flatten(y_pred[..., i])

        intersection = tf.keras.backend.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (
            tf.keras.backend.sum(y_true_class) + tf.keras.backend.sum(y_pred_class) + smooth
        )
        dice_scores.append(dice)

    return dice_scores  

In [12]:
def random_gamma(image):
    image = tf.clip_by_value(image, 1e-3, 1.0)
    gamma = tf.random.uniform([], 0.9, 1.1)
    image = tf.pow(image, gamma)
    return image

def augment(image, mask):
    image = tf.debugging.check_numerics(image, "Before augment")
    if tf.random.uniform(()) > 0.9:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
    if tf.random.uniform(()) > 0.9:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)
    if tf.random.uniform(()) > 0.6:
        image = random_gamma(image)
    return image, mask
def create_tf_dataset(images, masks, batch_size, augment_data):
        images = images.astype("float32") 
        masks = masks.astype("float32")  

        dataset = tf.data.Dataset.from_tensor_slices((images, masks))
        if augment_data:
            dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(len(images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset


strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    train_dataset = create_tf_dataset(X_train, Y_train, batch_size=8, augment_data=True)
    valid_dataset = create_tf_dataset(X_val, Y_val, batch_size=8, augment_data=False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [8]:
## LR schedule - not relevant

decay_steps = 40000
initial_learning_rate = 0.001
lr_schedule = CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    alpha=initial_learning_rate * 0.1
)

In [ ]:
def weighted_dice_loss(class_weights): # Experimental, will touch on later
    class_weights = tf.constant(class_weights, dtype=tf.float32)

    def loss_fn(y_true, y_pred, epsilon=1e-6):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        axes = [0, 1, 2] 
        intersection = tf.reduce_sum(y_true * y_pred, axis=axes)
        union = tf.reduce_sum(y_true + y_pred, axis=axes)
        dice = (2. * intersection + epsilon) / (union + epsilon)

        weighted_dice = class_weights * dice
        loss = 1. - tf.reduce_sum(weighted_dice)
        return loss

    return loss_fn

In [ ]:
with strategy.scope():
    tf.debugging.set_log_device_placement(False) 
    model = unet()
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss=dice_loss_per_class,
        metrics=[dice_metric]
    )
    
    checkpoint = ModelCheckpoint(
        filepath=f'models/best_unet2D_352_cropped',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=2
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.8, 
        patience=10,
        min_lr=0.00005,
        verbose=2
    )

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=200,
        callbacks=[checkpoint, reduce_lr]
    )

In [ ]:
with strategy.scope():    
    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=170,
        callbacks=[checkpoint, reduce_lr]
    )